Fossil Fuel Power Plants vs Renewable Power Plants

Comparing environmental(AQI), health (Chronic Resp) and economic data (CAINC1) by county.



- Compare Towns with Nuclear vs Towns without Nuclear
- Compare Towns with renewables within themselves
- Compare Towns with a power plant vs the rest of the state

Compare years 2000 and 2010


## Documentation

Power Plant Data: https://www.kaggle.com/datasets/behroozsohrabi/us-electric-power-plants/data

Resp Data: https://ghdx.healthdata.org/sites/default/files/record-attached-files/IHME_USA_COUNTY_RESP_DISEASE_MORTALITY_1980_2014_NATIONAL_XLSX.zip

AQI Data: https://aqs.epa.gov/aqsweb/airdata/annual_aqi_by_county_2010.zip, https://aqs.epa.gov/aqsweb/airdata/annual_aqi_by_county_2000.zip

Cainc1 Data: https://apps.bea.gov/regional/zip/CAINC1.zip

## Plan of Attack:
- Bring in all relevant data
- Filter for study population
    - Counties with Power Plants
- Join/Merge the different datasets into one master dataset for analysis
- Compare towns with power plants to towns without power plants
- Compare towns with different types of power plants
- Compare different types of renewables
- Discuss any confounding variables
- 

In [2]:

## Imports and constants
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

us_state_to_abbrev = {
    "ALABAMA": "AL",
    "ALASKA": "AK",
    "ARIZONA": "AZ",
    "ARKANSAS": "AR",
    "CALIFORNIA": "CA",
    "COLORADO": "CO",
    "CONNECTICUT": "CT",
    "DELAWARE": "DE",
    "FLORIDA": "FL",
    "GEORGIA": "GA",
    "HAWAII": "HI",
    "IDAHO": "ID",
    "ILLINOIS": "IL",
    "INDIANA": "IN",
    "IOWA": "IA",
    "KANSAS": "KS",
    "KENTUCKY": "KY",
    "LOUISANA": "LA",
    "MAINE": "ME",
    "MARYLAND": "MD",
    "MASSACHUSETTS": "MA",
    "MICHIGAN": "MI",
    "MINNESOTA": "MN",
    "MISSISSIPPI": "MS",
    "MISSOURI": "MO",
    "MONTANA": "MT",
    "NEBRASKA": "NE",
    "NEVADA": "NV",
    "NEW HAMPSHIRE": "NH",
    "NEW JERSEY": "NJ",
    "NEW MEXICO": "NM",
    "NEW YORK": "NY",
    "NORTH CAROLINA": "NC",
    "NORTH DAKOTA": "ND",
    "OHIO": "OH",
    "OKLAHOMA": "OK",
    "OREGON": "OR",
    "PENNSYLVANIA": "PA",
    "RHODE ISLAND": "RI",
    "SOUTH CAROLINA": "SC",
    "SOUTH DAKOTA": "SD",
    "TENNESSEE": "TN",
    "TEXAS": "TX",
    "UTAH": "UT",
    "VERMONT": "VT",
    "VIRGINIA": "VA",
    "WASHINGTON": "WA",
    "WEST VIRGINIA": "WV",
    "WISCONSIN": "WI",
    "WYOMING": "WY",
    "District of Columbia": "DC",
    "American Samoa": "AS",
    "Guam": "GU",
    "Northern Mariana Islands": "MP",
    "Puerto Rico": "PR",
    "United States Minor Outlying Islands": "UM",
    "U.S. Virgin Islands": "VI",
}   

In [67]:
## Init Power plant df (power_df)
power_df = pd.read_csv("data/Power_Plants.csv")
power_df = power_df[['CITY', 'STATE', 'COUNTY', 'NAICS_DESC', 'OPER_CAP', 'SUMMER_CAP', 'WINTER_CAP', 'PLAN_CAP',
       'RETIRE_CAP', 'GEN_UNITS', 'PLAN_UNIT', 'RETIR_UNIT', 'PRIM_FUEL',
       'SEC_FUEL', 'COAL_USED', 'NGAS_USED', 'OIL_USED', 'NET_GEN']] 
power_df.head()

,CITY,STATE,COUNTY,NAICS_DESC,OPER_CAP,SUMMER_CAP,WINTER_CAP,PLAN_CAP,RETIRE_CAP,GEN_UNITS,PLAN_UNIT,RETIR_UNIT,PRIM_FUEL,SEC_FUEL,COAL_USED,NGAS_USED,OIL_USED,NET_GEN
0,SAND POINT,AK,ALEUTIANS EAST,FOSSIL FUEL ELECTRIC POWER GENERATION,4.0,1.8,1.7,0.0,0.0,6,0,0,DFO,WND,0,0,0,347.00
1,NORTHPORT,AL,TUSCALOOSA,HYDROELECTRIC POWER GENERATION,53.9,53.0,53.0,0.0,0.0,1,0,0,WAT,NOT AVAILABLE,0,0,0,139170.00
2,BUCKS,AL,MOBILE,FOSSIL FUEL ELECTRIC POWER GENERATION,2569.5,2399.7,2430.7,774.0,272.0,10,2,1,NG,BIT,1292499,54793583,0,10499145.97
3,WETUMPKA,AL,ELMORE,HYDROELECTRIC POWER GENERATION,225.0,224.1,224.1,0.0,0.0,3,0,0,WAT,NOT AVAILABLE,0,0,0,554613.00
4,GADSDEN,AL,ETOWAH,FOSSIL FUEL ELECTRIC POWER GENERATION,138.0,130.0,130.0,0.0,0.0,2,0,0,NG,NOT AVAILABLE,0,697629,0,50435.00


In [13]:
# Init Respiratory Mortality Rate df (resp_df)
resp_df = pd.read_excel("data/RESP_DISEASE_MORTALITY_1980_2014.XLSX",header=1)
## Check the shape of the original data
display(resp_df.shape)
## Remove all NAs
resp_df.dropna(inplace=True)
resp_df['Location'] = resp_df['Location'].str.upper()
resp_df = resp_df[['Location','Mortality Rate, 2000*','Mortality Rate, 2010*']]
resp_df.columns = ['Location','Deaths per 100,000 in 2000','Deaths per 100,000 in 2010']
resp_df["County"] = resp_df["Location"].str.split(", ",expand=True)[0]
resp_df["State"]= resp_df["Location"].str.split(", ",expand=True)[1]
resp_df = resp_df.drop('Location',axis = 1)
resp_df = resp_df[['County','State','Deaths per 100,000 in 2000','Deaths per 100,000 in 2010']].dropna().reset_index(drop=True)
resp_df['State'] = resp_df['State'].map(us_state_to_abbrev)
loc_list = resp_df.head(0).copy()
for idx,row in power_df[['COUNTY','STATE']].iterrows():
    loc_list = pd.concat([loc_list,resp_df.loc[(resp_df['County'].str.contains(row['COUNTY'],case=False,regex=True)) & (resp_df['State'].str.contains(row['STATE'],case=False,regex=True))].reset_index(drop=True)])
resp_df = loc_list.copy()
resp_df = resp_df.drop_duplicates()
display(resp_df.shape)
resp_df.head()

(3196, 11)

(2181, 4)

,County,State,"Deaths per 100,000 in 2000","Deaths per 100,000 in 2010"
0,ALEUTIANS EAST BOROUGH,AK,"48.21 (41.35, 56.36)","35.45 (29.74, 42.07)"
0,TUSCALOOSA COUNTY,AL,"64.67 (62.02, 67.51)","67.32 (64.66, 70.10)"
0,MOBILE COUNTY,AL,"61.71 (59.69, 63.82)","63.00 (60.76, 65.35)"
0,ELMORE COUNTY,AL,"73.04 (69.48, 76.58)","84.14 (80.39, 88.57)"
0,ETOWAH COUNTY,AL,"77.87 (74.77, 81.02)","90.12 (86.34, 93.99)"


In [22]:
econ_df = pd.read_csv("data/CAINC1__ALL_AREAS_1969_2022.csv",encoding='latin-1')
econ_df.columns

Index(['GeoFIPS', 'GeoName', 'Region', 'TableName', 'LineCode',
       'IndustryClassification', 'Description', 'Unit', '1969', '1970', '1971',
       '1972', '1973', '1974', '1975', '1976', '1977', '1978', '1979', '1980',
       '1981', '1982', '1983', '1984', '1985', '1986', '1987', '1988', '1989',
       '1990', '1991', '1992', '1993', '1994', '1995', '1996', '1997', '1998',
       '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007',
       '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016',
       '2017', '2018', '2019', '2020', '2021', '2022'],
      dtype='object')

In [69]:
econ_df = pd.read_csv("data/CAINC1__ALL_AREAS_1969_2022.csv",encoding='latin-1')
econ_df = econ_df[['GeoName','Description','2000','2010']]
econ_df.dropna(inplace=True)
econ_df['GeoName'] = econ_df['GeoName'].str.upper()
econ_df
# for idx,row in power_df.iterrows():
#     loc_list = pd.concat([loc_list,econ_df.loc[(econ_df['GeoName'].str.contains(row['COUNTY'],case=False,regex=True)) & (econ_df['GeoName'].str.contains(row['STATE'],case=False,regex=True))].reset_index(drop=True)])
loc_list = pd.DataFrame(econ_df.head(0).copy())
for idx, row in power_df[['COUNTY','STATE']].drop_duplicates().iterrows():
    #if econ_df.loc[(econ_df['GeoName'].str.contains(row['COUNTY'],regex=True))]:
    loc_list = pd.concat([loc_list,econ_df.loc[(econ_df['GeoName'].str.contains(row['COUNTY'],case=False,regex=True)) & (econ_df['GeoName'].str.contains(row['STATE'],case=False,regex=True))].reset_index(drop=True)])
loc_list.drop_duplicates()


personal_income_df = loc_list[loc_list['Description'].str.contains('Personal income')]#.rename(columns = '')
personal_income_df = personal_income_df[['GeoName','2000','2010']].rename(columns = {'2000':'personal_income_2000','2010':'personal_income_2010'})

population_df = loc_list[loc_list['Description'].str.contains('Population')]
population_df = population_df[['GeoName','2000','2010']].rename(columns = {'2000':'pop_2000','2010':'pop_2010'})

per_capita_personal_income_df = loc_list[loc_list['Description'].str.contains('Per capita personal income')]
per_capita_personal_income_df = per_capita_personal_income_df[['GeoName','2000','2010']].rename(columns = {'2000':'per_capita_2000','2010':'per_capita_2010'})

final_econ_df = pd.merge(personal_income_df,population_df,how="inner",on='GeoName')
final_econ_df = pd.merge(final_econ_df,per_capita_personal_income_df,how="inner",on='GeoName')
final_econ_df.drop_duplicates(inplace=True)
display(final_econ_df.shape)

(2563, 3)

(2563, 3)

(2563, 3)

(2319, 7)

In [97]:
aqi2000_df = pd.read_csv("data/annual_aqi_by_county_2000.csv")
aqi2000_df.head()
# aqi2000_df['bad_percent'] = 100*(aqi2000_df['Unhealthy Days'].apply(float) + aqi2000_df['Very Unhealthy Days'].apply(float) + aqi2000_df['Hazardous Days'].apply(float))/(aqi2000_df['Days with AQI'].apply(float))
# aqi2010_df['bad_percent'] = 100*(aqi2010_df['Unhealthy Days'].apply(float) + aqi2010_df['Very Unhealthy Days'].apply(float) + aqi2010_df['Hazardous Days'].apply(float))/(aqi2010_df['Days with AQI'].apply(float))


,State,County,Year,Days with AQI,Good Days,Moderate Days,Unhealthy for Sensitive Groups Days,Unhealthy Days,Very Unhealthy Days,Hazardous Days,Max AQI,90th Percentile AQI,Median AQI,Days CO,Days NO2,Days Ozone,Days PM2.5,Days PM10
0,Alabama,Baldwin,2000,257,111,96,39,10,1,0,205,129,54,0,0,215,42,0
1,Alabama,Clay,2000,271,155,101,14,1,0,0,177,90,46,0,0,188,83,0
2,Alabama,Colbert,2000,106,38,66,2,0,0,0,124,80,58,0,0,0,106,0
3,Alabama,DeKalb,2000,354,169,123,58,4,0,0,159,115,51,0,0,297,56,1
4,Alabama,Elmore,2000,242,125,78,37,2,0,0,166,112,50,0,0,242,0,0


In [1]:
aqi2010_df = pd.read_csv("data/annual_aqi_by_county_2010.csv")
aqi2010_df.head()

NameError: name 'pd' is not defined